## Import Needed Libraries

In [ ]:
import argparse
import json
import os
from typing import List

import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
RACE_COLS = ["pct_white", "pct_black", "pct_asian", "pct_hispanic"]
ID_COL = "district_id"

In [ ]:
def build_features(df: pd.DataFrame, dop_race: bool) -> pd.DataFrame:
    cols = [c for c in df.columns if c!= ID_COL]
    if drop_race:
        cols = [c for c in cols if c not in RACE_COLS]
    return df[cols].copy(), cols